# PETSc training getting down to the cool kids

## Quick start
First install PETSc

In [1]:
!conda install -c conda-forge --yes petsc4py

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /home/nbuser/anaconda3_420:

The following NEW packages will be INSTALLED:

    blas:        1.1-openblas            conda-forge
    hypre:       2.11.2-1                conda-forge
    metis:       5.1.0-3                 conda-forge
    mpi:         1.0-mpich               conda-forge
    mpich:       3.2.1-1                 conda-forge
    mumps-mpi:   5.0.2-blas_openblas_0   conda-forge [blas_openblas]
    openblas:    0.2.20-8                conda-forge
    parmetis:    4.0.3-0                 conda-forge
    petsc:       3.9.1-blas_openblas_0   conda-forge [blas_openblas]
    petsc4py:    3.9.1-py35_0            conda-forge
    ptscotch:    6.0.4-6                 conda-forge
    readline:    7.0-ha6073c6_4                     
    scalapack:   2.0.2-1                 conda-forge
    scotch:      6.0.4-4                 conda-forge
    suitesparse: 4.5.4-blas_op

## Laplace equation

This example solves the Laplace equation,
$$-\Delta u = 1$$
using CG/Jacobi. This illustrates the use of custom Python operators, solvers, and preconditioners.

We start by importing the Python PETSc module and initialising. We are also importing example100 that has some utility stuff.

In [5]:
import petsc4py
from petsc4py import PETSc
import example100

petsc4py.init()

OptDB = PETSc.Options()
N     = OptDB.getInt('N', 100)
draw  = OptDB.getBool('draw', False)

# Setting up the sparse matrix

In [6]:
A = PETSc.Mat()
A.create(comm=PETSc.COMM_WORLD)
A.setSizes([N,N])
A.setType(PETSc.Mat.Type.PYTHON)
A.setPythonContext(example100.Laplace1D())
A.setUp()

In [4]:
x, b = A.getVecs()
b.set(1)

ksp = PETSc.KSP()
ksp.create(comm=PETSc.COMM_WORLD)
ksp.setType(PETSc.KSP.Type.PYTHON)
ksp.setPythonContext(example100.ConjGrad())

pc = ksp.getPC()
pc.setType(PETSc.PC.Type.PYTHON)
pc.setPythonContext(example100.Jacobi())

ksp.setOperators(A, A)
ksp.setFromOptions()
ksp.solve(b, x)

r = b.duplicate()
A.mult(x, r)
r.aypx(-1, b)
rnorm = r.norm()
print(rnorm)
PETSc.Sys.Print('error norm = %g' % rnorm, comm=PETSc.COMM_WORLD)

if draw:
    viewer = PETSc.Viewer.DRAW(x.getComm())
    x.view(viewer)
    PETSc.Sys.sleep(2)

6.700822229941938e-13
